In [1]:
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from gensim.test.utils import datapath

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use',])
from nltk.tokenize import word_tokenize
import pandas as pd

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ShriyaChepur/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ShriyaChepur/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
import nltk
nltk.download('wordnet')

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ShriyaChepur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df = pd.read_csv('common_products_parsed.csv')

In [8]:
df.drop(labels=['user_id','category'],axis=1,inplace=True)


In [9]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [10]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text,n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [11]:
df['docs'] = df['docs'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df = df.fillna('')
df['docs']= df['docs'].apply(lambda x:remove_punctuation(x))
df['docs']= df['docs'].apply(lambda x: x.lower())

In [11]:
#df['keywords']=''
#df['trigrams'] = df['docs'].apply(lambda row:list(nltk.ngrams(row.split(), 3)))

In [23]:
#dff=df.copy()

In [12]:
df['trigrams'] = df['docs'].apply(lambda text:get_ngrams(text,3))

In [35]:
#dff['doc'] = dff.docs.apply(lemmatize_text)

In [13]:
id2word = corpora.Dictionary(df['trigrams'])

In [14]:
texts = df['trigrams']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word, num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
temp_file = datapath("model")

print("saving model to " + temp_file, flush=True)

lda_model.save(temp_file)

saving model to /Applications/anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model


In [16]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(21,
  '0.000*"care either understand" + 0.000*"bragging prints fade" + 0.000*"come '
  'moneyback guarantee" + 0.000*"come alive eyes" + 0.000*"choices help '
  'memorable" + 0.000*"cool holiday shot" + 0.000*"best gift boyfriend" + '
  '0.000*"best gift husband" + 0.000*"design choices help" + 0.000*"boyfriend '
  'shot glasses"'),
 (38,
  '0.000*"care either understand" + 0.000*"bragging prints fade" + 0.000*"come '
  'moneyback guarantee" + 0.000*"come alive eyes" + 0.000*"choices help '
  'memorable" + 0.000*"cool holiday shot" + 0.000*"best gift boyfriend" + '
  '0.000*"best gift husband" + 0.000*"design choices help" + 0.000*"boyfriend '
  'shot glasses"'),
 (97,
  '0.000*"care either understand" + 0.000*"bragging prints fade" + 0.000*"come '
  'moneyback guarantee" + 0.000*"come alive eyes" + 0.000*"choices help '
  'memorable" + 0.000*"cool holiday shot" + 0.000*"best gift boyfriend" + '
  '0.000*"best gift husband" + 0.000*"design choices help" + 0.000*"boyfriend '
  'shot g

In [18]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=texts):
    sent_topics_df = pd.DataFrame()
    #print(ldamodel[corpus])
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list      
        #print(ldamodel)
        row = sorted(row, kbey=lambda x: (x[1]), reverse=True)
        #print(row)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: 
                wp = ldamodel.show_topic(topic_num)
                #print(wp)
                #print(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,75.0,0.9635,"mikrofon und menï¿½tasten, board platine mikro...","[ngmobile usb lade, usb lade buchse, lade buch..."
1,1,34.0,0.9236,"spinmaster deathgripper hidden, misterioso cã³...","[spinmaster deathgripper hidden, deathgripper ..."
2,2,72.0,0.9903,"epson powerlite epson, powerlite epson powerli...","[emazne projector replacement, projector repla..."
3,3,61.0,0.0115,"original classics sterling, sterling silver ri...","[lavello plados spazio, plados spazio due, spa..."
4,4,61.0,0.0115,"original classics sterling, sterling silver ri...","[dan post boot, post boot company, boot compan..."
5,5,61.0,0.9889,"original classics sterling, sterling silver ri...","[original classics sterling, classics sterling..."
6,6,53.0,0.9889,"wooden brass poles, fixed end socket, supports...","[gorgeous girlfriend mug, girlfriend mug black..."
7,7,70.0,0.9956,"kenya lo prepariamo, caffe macinato di, cattur...","[viaggio espresso caffã¨, espresso caffã¨ maci..."
8,8,74.0,0.9649,"organizador de viaje, cubo diseã±o ãºnico, cal...","[cubos de equipaje, de equipaje de, equipaje d..."
9,9,53.0,0.9821,"wooden brass poles, fixed end socket, supports...","[deco interior designing, interior designing x..."


In [19]:
df_dominant_topic.drop(labels=['Document_No'],inplace=True,axis=1)

In [20]:
df_dominant_topic.head(10)

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,75.0,0.9635,"mikrofon und menï¿½tasten, board platine mikro...","[ngmobile usb lade, usb lade buchse, lade buch..."
1,34.0,0.9236,"spinmaster deathgripper hidden, misterioso cã³...","[spinmaster deathgripper hidden, deathgripper ..."
2,72.0,0.9903,"epson powerlite epson, powerlite epson powerli...","[emazne projector replacement, projector repla..."
3,61.0,0.0115,"original classics sterling, sterling silver ri...","[lavello plados spazio, plados spazio due, spa..."
4,61.0,0.0115,"original classics sterling, sterling silver ri...","[dan post boot, post boot company, boot compan..."
5,61.0,0.9889,"original classics sterling, sterling silver ri...","[original classics sterling, classics sterling..."
6,53.0,0.9889,"wooden brass poles, fixed end socket, supports...","[gorgeous girlfriend mug, girlfriend mug black..."
7,70.0,0.9956,"kenya lo prepariamo, caffe macinato di, cattur...","[viaggio espresso caffã¨, espresso caffã¨ maci..."
8,74.0,0.9649,"organizador de viaje, cubo diseã±o ãºnico, cal...","[cubos de equipaje, de equipaje de, equipaje d..."
9,53.0,0.9821,"wooden brass poles, fixed end socket, supports...","[deco interior designing, interior designing x..."


In [22]:
df.drop(labels=['trigrams'],inplace=True,axis=1)

In [23]:
df

,sku,market_place,product_id,docs
0,1004586,A1PA6795UKMFR9,B0185SP7HY,ngmobile usb lade buchse docking port flex kab...
1,81-D3KN-HRAQ,A1AM78C64UM0Y8,B07PH3QFXC,spinmaster deathgripper hidden world dragon pu...
2,XIIX-ELPLP22-04,ATVPDKIKX0DER,B073LP2WMK,emazne projector replacement compatible lamp h...
3,SP116295,APJ6JRA9NG5V4,B01F7E47GQ,lavello plados spazio due vasche piã¹ scolapiatti
4,D-DP69512-15W,A13V1IB3VIYZZH,B004GZFXZW,dan post boot company homme marron saddle eu
...,...,...,...,...
95,AT-0Q4W-MN52,ATVPDKIKX0DER,B0794VWL3L,paas easter egg dying kits eggheads
96,066-White-36,SHOPIFY,6652433301590,womens flyknit tennis laceup sneakers
97,O0109DCKUS0762052494717,A1AM78C64UM0Y8,B00ATUYGR4,moving comfort womens maia bra black sports bra
98,PW-ADAP017-HP EliteBook 840 G3,A1AM78C64UM0Y8,B08LMGRXLR,cargador adaptador compatible para hp eliteboo...


In [24]:
result = pd.concat([df, df_dominant_topic], axis=1, ignore_index=False)

In [27]:
result.drop(labels=['Text'],inplace=True,axis=1)

In [28]:
result.head()

,sku,market_place,product_id,docs,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,1004586,A1PA6795UKMFR9,B0185SP7HY,ngmobile usb lade buchse docking port flex kab...,75.0,0.9635,"mikrofon und menï¿½tasten, board platine mikro..."
1,81-D3KN-HRAQ,A1AM78C64UM0Y8,B07PH3QFXC,spinmaster deathgripper hidden world dragon pu...,34.0,0.9236,"spinmaster deathgripper hidden, misterioso cã³..."
2,XIIX-ELPLP22-04,ATVPDKIKX0DER,B073LP2WMK,emazne projector replacement compatible lamp h...,72.0,0.9903,"epson powerlite epson, powerlite epson powerli..."
3,SP116295,APJ6JRA9NG5V4,B01F7E47GQ,lavello plados spazio due vasche piã¹ scolapiatti,61.0,0.0115,"original classics sterling, sterling silver ri..."
4,D-DP69512-15W,A13V1IB3VIYZZH,B004GZFXZW,dan post boot company homme marron saddle eu,61.0,0.0115,"original classics sterling, sterling silver ri..."


In [30]:
result['Keywords']

0     mikrofon und menï¿½tasten, board platine mikro...
1     spinmaster deathgripper hidden, misterioso cã³...
2     epson powerlite epson, powerlite epson powerli...
3     original classics sterling, sterling silver ri...
4     original classics sterling, sterling silver ri...
                            ...                        
95    original classics sterling, sterling silver ri...
96    original classics sterling, sterling silver ri...
97    original classics sterling, sterling silver ri...
98    contacto con nosotros, comprar si tiene, que s...
99    enhance beauty regular, full sleeves denim, mb...
Name: Keywords, Length: 100, dtype: object

In [ ]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()

In [ ]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()
session=clstr.connect('db')

In [ ]:
qry= '''
create table common_products_parsed_keywords
 (
   sku text,
   market_place text,
   product_id text,
   docs text,
   Dominant_Topic float,
   Topic_Perc_Contrib float,
   keywords text,
   primary key(product_id)
);'''
session.execute(qry)

In [ ]:
qr="INSERT INTO common_products_parsed_keywords(sku,market_place,product_id,docs,Dominant_Topic,Topic_Perc_Contrib,keywords) VALUES (?,?,?,?,?,?,?)"
prepared = session.prepare(qr)

In [ ]:
for i,j,k,l,m,n,o in zip(x['sku'],x['market_place'],x['product_id'],x["docs"],x['Dominant_Topic'],x['Topic_Perc_Contrib'],x['keywords']):
     session.execute(prepared, (i,j,k,l,m))